In [1]:
import pandas as pd

# Load datasets
customers = pd.read_csv("olist_customers_dataset.csv")
geolocation = pd.read_csv("olist_geolocation_dataset.csv")
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
category_translation = pd.read_csv("product_category_name_translation.csv")

In [2]:
orders_customers = pd.merge(orders, customers, on='customer_id', how='left')


In [3]:
orders_items = pd.merge(orders_customers, order_items, on='order_id', how='left')

In [4]:
orders_items_products = pd.merge(orders_items, products, on='product_id', how='left')
orders_items_products = pd.merge(orders_items_products, category_translation, on='product_category_name', how='left')

In [5]:
orders_with_sellers = pd.merge(orders_items_products, sellers, on='seller_id', how='left')

In [6]:
orders_with_payments = pd.merge(orders_with_sellers, payments, on='order_id', how='left')
olist_master_df = pd.merge(orders_with_payments, reviews, on='order_id', how='left')

In [7]:
date_cols = [
    'order_purchase_timestamp', 
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date',
    'order_estimated_delivery_date',
    'review_creation_date',
    'review_answer_timestamp'
]

for col in date_cols:
    olist_master_df[col] = pd.to_datetime(olist_master_df[col], errors='coerce', dayfirst=True)

In [8]:
# Delivery performance
olist_master_df["delivery_delay_days"] = (
    olist_master_df["order_delivered_customer_date"] - olist_master_df["order_estimated_delivery_date"]
).dt.days

olist_master_df["actual_delivery_days"] = (
    olist_master_df["order_delivered_customer_date"] - olist_master_df["order_purchase_timestamp"]
).dt.days

# Time to approve order
olist_master_df["approval_time_hours"] = (
    olist_master_df["order_approved_at"] - olist_master_df["order_purchase_timestamp"]
).dt.total_seconds() / 3600

# Time to ship after approval
olist_master_df["shipping_delay_days"] = (
    olist_master_df["order_delivered_carrier_date"] - olist_master_df["order_approved_at"]
).dt.days

These fields will help you answer business questions like:

Are we shipping late?

How fast do we approve orders?

How long does delivery actually take?



In [9]:
# Fill NaNs in delay columns with 0 or reasonable fallback
olist_master_df["delivery_delay_days"].fillna(0, inplace=True)
olist_master_df["actual_delivery_days"].fillna(0, inplace=True)
olist_master_df["approval_time_hours"].fillna(0, inplace=True)
olist_master_df["shipping_delay_days"].fillna(0, inplace=True)

C:\Users\newto\AppData\Local\Temp\ipykernel_13452\1459055427.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  olist_master_df["delivery_delay_days"].fillna(0, inplace=True)
C:\Users\newto\AppData\Local\Temp\ipykernel_13452\1459055427.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a 

In [10]:
olist_master_df.to_csv("olist_master_cleaned.csv", index=False)

In [11]:
olist_master_df.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,delivery_delay_days,actual_delivery_days,approval_time_hours,shipping_delay_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,2017-10-02 11:07:00,2017-10-04 19:55:00,2017-10-10 21:25:00,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:00,-8.0,8.0,0.183333,2.0
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,2017-10-02 11:07:00,2017-10-04 19:55:00,2017-10-10 21:25:00,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:00,-8.0,8.0,0.183333,2.0
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:00,2017-10-02 11:07:00,2017-10-04 19:55:00,2017-10-10 21:25:00,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,...,a54f0611adc9ed256b57ede6b6eb5114,4.0,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:00,-8.0,8.0,0.183333,2.0
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:00,2018-07-26 03:24:00,2018-07-26 14:31:00,2018-08-07 15:27:00,2018-08-13,af07308b275d755c9edb36a90c618231,47813,...,8d5266042046a06655c8db133d120ba5,4.0,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08 18:37:00,-6.0,13.0,30.716667,0.0
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:00,2018-08-08 08:55:00,2018-08-08 13:50:00,2018-08-17 18:06:00,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,e73b67b67587f7644d5bd1a52deb1b01,5.0,NaN,NaN,2018-08-18,2018-08-22 19:07:00,-18.0,9.0,0.283333,0.0
